## Kaggle Housing Prices Competition

### Prepare the data for machine learning algorithms
* Deal with missing values
* Handle outliers
* Deal with correlated attributes
* Handle text and categorical attributes
* Feature scaling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load data 
housing = pd.read_csv('housing_price_data/train.csv')
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


#### Address Missing Values

In [5]:
housing.isnull().sum().sort_values(ascending = False)

PoolQC         1453
MiscFeature    1406
Alley          1369
Fence          1179
MasVnrType      872
               ... 
ExterQual         0
Exterior2nd       0
Exterior1st       0
RoofMatl          0
SalePrice         0
Length: 81, dtype: int64

In [6]:
# Imputing Missing Values
housing_processed = housing

# Categorical columns:
cat_cols_fill_none = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                     'GarageCond', 'GarageQual', 'GarageFinish', 'GarageType',
                     'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond',
                     'MasVnrType']

# Replace missing values for categorical columns with None
for cat in cat_cols_fill_none:
    housing_processed[cat] = housing_processed[cat].fillna("None")
    
# Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
housing_processed['LotFrontage'] = housing_processed.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))    

# Garage: GarageYrBlt, GarageArea and GarageCars these are numerical columns, replace with zero
for col in ['GarageYrBlt', 'GarageArea', 'GarageCars']:
    housing_processed[col] = housing_processed[col].fillna(int(0))
    
# MasVnrArea : replace with zero
housing_processed['MasVnrArea'] = housing_processed['MasVnrArea'].fillna(int(0))

# Use the mode value 
housing_processed['Electrical'] = housing_processed['Electrical'].fillna(housing_processed['Electrical']).mode()[0]

# There is no need of Utilities so let's just drop this column
housing_processed = housing_processed.drop(['Utilities'], axis = 1)

# Get the count again to verify that we do not have any more missing values
housing_processed.isnull().apply(sum).max()

0

#### Address Outliers

In [9]:
# Select only numeric attributes
num_features = housing_processed.select_dtypes(exclude = 'object')

# Calculate the 99.9th quantile for numeric attributes
high_quant = num_features.quantile(0.999)

# Remove outliers
for i in num_features.columns:
    # Drop rows where the value exceeds the high quantile
    housing_processed = housing_processed.drop(housing_processed[housing_processed[i] > high_quant[i]].index)

# Check the resulting DataFrame info
housing_processed.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1421 entries, 0 to 1457
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1421 non-null   int64  
 1   MSSubClass     1421 non-null   int64  
 2   MSZoning       1421 non-null   object 
 3   LotFrontage    1421 non-null   float64
 4   LotArea        1421 non-null   int64  
 5   Street         1421 non-null   object 
 6   Alley          1421 non-null   object 
 7   LotShape       1421 non-null   object 
 8   LandContour    1421 non-null   object 
 9   LotConfig      1421 non-null   object 
 10  LandSlope      1421 non-null   object 
 11  Neighborhood   1421 non-null   object 
 12  Condition1     1421 non-null   object 
 13  Condition2     1421 non-null   object 
 14  BldgType       1421 non-null   object 
 15  HouseStyle     1421 non-null   object 
 16  OverallQual    1421 non-null   int64  
 17  OverallCond    1421 non-null   int64  
 18  YearBuilt    

#### Address Correlated Features

In [10]:
# Remove highly correlated features
feature_drop = ['MiscVal', 'MoSold', 'YrSold', 'BsmtFinSF2','BsmtHalfBath','MSSubClass',
                   'GarageArea', 'GarageYrBlt', '3SsnPorch']

housing_processed = housing_processed.drop(feature_drop, axis = 1)

#### Transform Categorical Features (One-Hot Encode)

In [11]:
# Transforming categorical features
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_processed_1hot = cat_encoder.fit_transform(housing_processed)
housing_processed_1hot

<1421x8750 sparse matrix of type '<class 'numpy.float64'>'
	with 100891 stored elements in Compressed Sparse Row format>

#### Feature Scaling

In [14]:
# Import modules
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Separate features and target variable
housing_X = housing_processed.drop("SalePrice", axis = 1)
housing_y = housing_processed["SalePrice"].copy()

# Get the list of names for numerical and categorical attributes separately
num_features = housing_X.select_dtypes(exclude = 'object')
cat_features = housing_X.select_dtypes(include = 'object')

num_feats = list(num_attributes)
cat_feats = list(cat_attributes)

# Numerical Pipeline to impute any missing values with the median and scale attributes
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy = "median")),
        ('std_scaler', StandardScaler()),
    ])

In [15]:
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_feats),
        ("cat", OneHotEncoder(), cat_feats),
    ])


# Description before applying transforms
print(housing_y.describe())

# Apply log-transform to SalePrice
housing_y_prepared  = np.log(housing_y)

# Run the transformation pipeline on all the other attributes
housing_X_prepared = full_pipeline.fit_transform(housing_X)

# Description before applying transforms
print(housing_y_prepared.describe())

housing_X_prepared

count      1421.000000
mean     178430.574243
std       74526.933464
min       35311.000000
25%      129500.000000
50%      161500.000000
75%      212000.000000
max      611657.000000
Name: SalePrice, dtype: float64
count    1421.000000
mean       12.014898
std         0.389710
min        10.471950
25%        11.771436
50%        11.992260
75%        12.264342
max        13.323927
Name: SalePrice, dtype: float64


<1421x282 sparse matrix of type '<class 'numpy.float64'>'
	with 99470 stored elements in Compressed Sparse Row format>

In [18]:
# Save Preprocessed data sets to be used in Machine Learning model
import pickle
import os

# Define the folder path
folder_path = 'housing_price_data_processed'
os.makedirs(folder_path, exist_ok = True)  # Create the folder if it doesn't exist

# Save the data
with open(os.path.join(folder_path, 'housing_X_prepared.pkl'), 'wb') as f:
    pickle.dump(housing_X_prepared, f)

with open(os.path.join(folder_path, 'housing_y_prepared.pkl'), 'wb') as f:
    pickle.dump(housing_y_prepared, f)